# Adding a Convolutional Layer to the Feedforward Neural Network from HW2

To make things easier, rather than include a different CNN for problems 2,3,4,and 5, I have included a singular CNN that has all of the options for those problems included, and they can easily be adjusted or turned on or off at initialization.

Reading in the data

In [4]:
import numpy as np
import struct



def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)
    
X_train = read_idx('train-images-idx3-ubyte')
y_train = read_idx('train-labels-idx1-ubyte')
X_test = read_idx('t10k-images-idx3-ubyte')
y_test = read_idx('t10k-labels-idx1-ubyte')


## Scaling the values so that the model can more accurately adjust the weights
X_train = X_train.astype(np.float)/255.
X_test = X_test.astype(np.float)/255.

2. Organic CNN Replace the fully connected layer from the first problem with the convolutional layer.

3. Initialization Implement four different initialization techniques and apply them to your CNN model:
    - Zeros initialization
    - Random initialization. This initializes the weights to some random values.
    - Xavier initialization, which scales the variance of the inputs to each layer are scaled to have variance of sqrt(1./layers_dims[l-1]).
    - He initialization. This initializes the weights to random values scaled according to a paper by He et al.,
    2015. Similar to Xavier you need to scale the inputs so they have the variance sqrt(2./layers_dims[l-1])

4. Optimization Implement ADAM modification to the SGD and apply it to MNIST classification.

5. Dropout Implement forward and backward propagation with dropout. Apply to MNIST classification problem.

In [5]:
class CNN_Weight_Adam_dropout:
    """ Defining a Convolutional Neural Network Model
        with l2 regularization
    
    The parameters are: 
        l2: l2 regularization rate
        n_iter: number of iterations over the dataset
        eta: learning rate/step value
        hidden_act: activation function that is used on the hidden layer
        encode: if the target values are not in binary form, then this 
        will one hot encode them
        batch_size: number of samples used in each batch
        number_kernels: the depth of kernels in the convolutional layer
        filter_size: the size of the kernel filter
    """

    
    def __init__(self, n_iter=1000, eta=0.05, hidden_act='sigmoid',
                 batch_size=100, number_kernels=8, filter_size=3,
                 weight_method='random', dropout_prob=0.0):
        self.n_iter = n_iter
        self.eta = eta
        self.batch_size = batch_size
        self.hidden_act = hidden_act
        self.number_kernels = number_kernels
        self.filter_size = filter_size
        self.weight_method = weight_method
        self.dropout_prob = dropout_prob
            
    def hidden_layer(self, z):
        # defining which activation function gets used in the hidden layer
        # adding an upper limit to the relu values to prevent the gradient from 
        # exploding
        if self.hidden_act=='tanh':
            return (np.exp(z) - np.exp(-z))/(np.exp(z) + np.exp(-z))
        else:
            return 1/(1+ np.exp(-z))

    
    def softmax(self, z):
        # This is the softmax function of z_out
        # used to get the predicted class for a multi-class problem
        # scaling the softmax results to prevent values from converging to nan
        z = z- np.max(z)
        return np.exp(z)/np.sum(np.exp(z), axis=1).reshape(-1,1)
    
    def onehot(self, data):
        """ This takes in target values and returns them coded in binary for each
            distinct class. The index values don't necessarily correspond to the
            original target values as the values are mapped into a distinct range.
        """
        n_classes = np.unique(data)
        onehot_array = np.zeros((data.shape[0], np.max(data)+1))
        onehot_array[np.arange(data.shape[0]), data] = 1
        if n_classes.shape[0] > 2:
            return onehot_array[:,n_classes]     
        # return only one column if there are only two target variable to avoid
        # redundancy. This reflects the first instance (ordinal) in the data.
        else:
            return onehot_array[:,n_classes[0]]
        
    def iterate_kernels(self, X,R,C, batch, filter_size, step_size):
        for i in np.arange(0,self.output_size-(filter_size-1),step_size):
            for j in np.arange(0, self.output_size-(filter_size-1), step_size):
                yield X[batch,i:i+filter_size,j:j+filter_size], i, j     
                
    def forward_prop(self, X, R, C, filter_size, step_size, number_kernels,
                     output_size, batch_size):
        # forward propogation through the neural network
        Z_conv = np.zeros((batch_size, output_size, output_size, self.number_kernels)) 
        for batch in range(batch_size):
            for X_region, i, j in self.iterate_kernels(X,R,C,batch, filter_size, step_size):
                Z_conv[batch, i,j] = np.sum(X_region * self.Wconv) + self.Bconv
        
        
        A_h = self.hidden_layer(Z_conv)
        # Using Dropout
        A_h = self.dropout(A_h, self.dropout_prob)
        
        
        # softmax layer
        flat_A_h = A_h.reshape(batch_size,self.flat_weight)
        
        Z_out = np.dot(flat_A_h, self.Wout) + self.Bout
        A_out = Z_out
        A_out = self.softmax(A_out)
        
        return Z_conv, A_h, Z_out, A_out


    def backprop(self, Z_conv, A_h, Z_out, A_out, X_train, y_train):       
        # [num in X, n classes]
        output_error = A_out - y_train
        
        # [n hidden units, num in X] dot [num in X, n classes]
        # -> [n hidden units, n classes]
        DJ_DWout = np.dot(A_h.reshape(self.batch_size, self.flat_weight).T, output_error)
        
        # [num in X, n hidden units] 
        # The derivatives of the activation functions are different
        if self.hidden_act == 'tanh':
            hidden_deriv = 1 - A_h**2

        else:
            hidden_deriv = A_h * (1-A_h)
        
        # [num in X, n classes] dot [n classes, num hidden units]
        # * [num in X, n hidden units] -> [num in X, n hidden units]
        DJ_DWhid = np.dot(output_error, self.Wout.T).reshape(self.batch_size,self.output_size,self.output_size,self.number_kernels) * hidden_deriv
        
        
        filters_deriv = np.zeros(self.Wconv.shape)
        
        for batch in range(self.batch_size):
            for X_region, i, j in self.iterate_kernels(X_train, X_train.shape[1],X_train.shape[2], batch, self.filter_size,1):
                for f in range(self.number_kernels):
                    filters_deriv[f] += np.sum(DJ_DWhid[batch,i,j,f] * X_region)
        
        
        # getting the gradients into the number of filters
        grad_Wconv = filters_deriv
        grad_Bconv = np.sum(DJ_DWhid, axis=(0,1,2))
        
        # getting the gradients into the number of batches
        grad_Wout = DJ_DWout
        grad_Bout = np.sum(output_error, axis=0)
        

        
        return grad_Wconv, grad_Bconv, grad_Wout, grad_Bout
        
    def weight_init_(self, rows, cols, method, three_D=False, third_dim=None):
        
        if three_D:
            # Zero initialization
            if method == 'zero':
                x = np.zeros((third_dim, rows, cols))
            # Xavier initialization
            elif method == 'Xavier':
                x = np.zeros((third_dim, rows, cols))
                for i in range(third_dim):
                    x[i] = np.random.randn(rows, cols)*np.sqrt(1/cols)
            # He initialization
            elif method == 'He':
                x = np.zeros((third_dim, rows, cols))
                for i in range(third_dim):
                    x[i] = np.random.randn(rows, cols)*np.sqrt(2/cols)
            # Random initialization
            else:
                x = np.random.randn((third_dim, rows, cols))
        else:
            # Zero initialization
            if method == 'zero':
                x = np.zeros((rows, cols))
            # Xavier initialization
            elif method == 'Xavier':
                x = np.random.randn(rows, cols)*np.sqrt(1/cols)
            # He initialization
            elif method == 'He':
                x = np.random.randn(rows, cols)*np.sqrt(2/cols)
            # Random initialization
            else:
                x = np.random.randn(rows, cols)
        return x    

    ## Adam optimization
    def adam(self, gradient, t, w,m,v,learning_rate=0.05, beta_1=0.9, beta_2=0.999, epsilon=1e-8):
            m = beta_1 * m + (1-beta_1)*gradient
            v = beta_2 * v + (1-beta_2)*(gradient**2)
            m_hat = m/(1-beta_1**t)
            v_hat = v/(1-beta_2**t)
            w = w - learning_rate*(m_hat/(np.sqrt(v_hat) + epsilon))
            return w, m, v
    
    ## Dropout
    def dropout(self, data, dropout_prob):
        percentage_kept = 1-dropout_prob
        
        # getting the values to keep and scaling them
        self.mask = np.random.binomial(1,percentage_kept,size=data.shape)*(percentage_kept)
        
        return self.mask * data
    
    def fit(self, X_train, y_train, X_valid, y_valid):
        

        y_train_oh = self.onehot(y_train)
        # forward propogation through the neural network
        padding = 0
        step_size=1
        R, C = X_train.shape[1:3]
        num_classes = 10
        self.output_size = int(((R - self.filter_size + 2*padding)/step_size) + 1)
        self.flat_weight = self.output_size*self.output_size*self.number_kernels
        self.Wconv = self.weight_init_(self.filter_size,self.filter_size,self.weight_method,
                                      three_D=True, third_dim=self.number_kernels)
        self.Bconv = np.zeros(self.number_kernels) 
        self.Wout = self.weight_init_(self.flat_weight, num_classes,self.weight_method)
        self.Bout = np.zeros(num_classes)
        
        # initialize the adam optimization parameters
        self.Wconv_m, self.Wconv_v = 0,0
        self.Bconv_m, self.Bconv_v = 0,0
        self.Wout_m, self.Wout_v = 0,0
        self.Bout_m, self.Bout_v = 0,0
        m = y_train.shape[0]
        
        time = 0
        self.metrics = {'cost': [],'train_accuracy': [], 'valid_accuracy':[]}
        for epoch in range(self.n_iter):
            print("Epoch: {}".format(epoch+1))
            shuffled_values = np.random.permutation(m)
            X_shuffled = X_train[shuffled_values]
            y_shuffled = y_train_oh[shuffled_values]
            for batch in range(0,m,self.batch_size):
                time += 1
                print("Batch {:.0f}/{:.0f}".format((batch/self.batch_size)+1,m/self.batch_size))
                x_batch = X_shuffled[batch:batch+self.batch_size]
                y_batch = y_shuffled[batch:batch+self.batch_size]
                
                # Forward Propagation
                Z_conv, A_h, Z_out, A_out = self.forward_prop(x_batch,R,C,self.filter_size,
                                                       step_size, self.number_kernels,
                                                       self.output_size, self.batch_size)
                
                # Backpropagation
                grad_Wconv, grad_Bconv, grad_Wout, grad_Bout = self.backprop(Z_conv, A_h, Z_out, A_out, 
                                                                             x_batch, 
                                                                             y_batch)
            
                ## Adam optimization
                self.Wconv, self.Wconv_m, self.Wconv_v = self.adam(grad_Wconv,t = time,w=self.Wconv, 
                                                          m=self.Wconv_m, v=self.Wconv_v, learning_rate=self.eta)
                self.Bconv, self.Bconv_m, self.Bconv_v = self.adam(grad_Bconv, t= time, 
                                                                    w=self.Bconv, m=self.Bconv_m, v=self.Bconv_v,
                                                                    learning_rate=self.eta)
        
                self.Wout, self.Wout_m, self.Wout_v = self.adam(grad_Wout, t=time, w=self.Wout, 
                                                                m=self.Wout_m, v=self.Wout_v,
                                                                learning_rate=self.eta)
                self.Bout, self.Bout_m, self.Bout_v = self.adam(grad_Bout, t=time, 
                                                                      w=self.Bout, m=self.Bout_m, v=self.Bout_v,
                                                                      learning_rate=self.eta)
                

        
        
            # After each iteration, do an evaluation
            # Evaluating on the training set
            Z_conv, A_h, Z_out, A_out = self.forward_prop(X_train,R,C,self.filter_size,
                                                   step_size, self.number_kernels,
                                                   self.output_size, batch_size=m)
            
            cost = self.cost_function(A_out, y_train_oh)
            train_predictions = self.predict(A_out)
            
            # Evaluation on the validation set
            Z_conv, A_h, Z_out, A_out = self.forward_prop(X_valid,R,C,self.filter_size,
                                                   step_size, self.number_kernels,
                                                   self.output_size, batch_size=y_valid.shape[0])
            
            valid_predictions = self.predict(A_out)
            
            
    
            train_accuracy = (np.sum(train_predictions == y_train).astype(np.float)/train_predictions.shape[0])*100
            valid_accuracy = (np.sum(valid_predictions == y_valid).astype(np.float)/valid_predictions.shape[0])*100

            print("Epoch: {}\t Train Acc: {:.3f}\t Validation Acc: {:.3f}".format(epoch+1, train_accuracy, valid_accuracy))
            
            self.metrics['cost'].append(cost)
            self.metrics['train_accuracy'].append(train_accuracy)
            self.metrics['valid_accuracy'].append(valid_accuracy)

    def cost_function(self, A_out, y):
        self.A_out = A_out
        self.y_oh = y
        return np.average(np.sum((-y*np.log(A_out)),axis=1))
    
    def predict(self, output):
        # return the value with the highest percentage
        return np.argmax(output, axis=1)
    

In [6]:
X_train_sample = X_train[:2000]
y_train_sample = y_train[:2000]
     
X_train_valid = X_train[40000:40500]
y_train_valid = y_train[40000:40500]
    
    
model = CNN_Weight_Adam_dropout(n_iter=3, eta=0.05, hidden_act = 'tanh',
                 batch_size=100, number_kernels=12, filter_size=3,
                 weight_method='Xavier', dropout_prob=0.0)

model.fit(X_train_sample, y_train_sample, X_train_valid, y_train_valid)

Epoch: 1
Batch 1/20
Batch 2/20
Batch 3/20
Batch 4/20
Batch 5/20
Batch 6/20
Batch 7/20
Batch 8/20
Batch 9/20
Batch 10/20
Batch 11/20
Batch 12/20
Batch 13/20
Batch 14/20
Batch 15/20
Batch 16/20
Batch 17/20
Batch 18/20
Batch 19/20
Batch 20/20
Epoch: 1	 Train Acc: 85.500	 Validation Acc: 80.800
Epoch: 2
Batch 1/20
Batch 2/20
Batch 3/20
Batch 4/20
Batch 5/20
Batch 6/20
Batch 7/20
Batch 8/20
Batch 9/20
Batch 10/20
Batch 11/20
Batch 12/20
Batch 13/20
Batch 14/20
Batch 15/20
Batch 16/20
Batch 17/20
Batch 18/20
Batch 19/20
Batch 20/20
Epoch: 2	 Train Acc: 90.050	 Validation Acc: 83.800
Epoch: 3
Batch 1/20
Batch 2/20
Batch 3/20
Batch 4/20
Batch 5/20
Batch 6/20
Batch 7/20
Batch 8/20
Batch 9/20
Batch 10/20
Batch 11/20
Batch 12/20
Batch 13/20
Batch 14/20
Batch 15/20
Batch 16/20
Batch 17/20
Batch 18/20
Batch 19/20
Batch 20/20
Epoch: 3	 Train Acc: 90.900	 Validation Acc: 80.200
